In [1]:
from os import getenv
from datetime import datetime, timedelta
from math import degrees
import ephem
from dotenv import load_dotenv

load_dotenv();

In [2]:
BALCONY_LAT = getenv('BALCONY_LAT')
BALCONY_LON = getenv('BALCONY_LON')

# This is not my balcony's elevation, but the elevation of my city above sea level.
# This is due to the way ephem treats an observer's elevation to be the same as the 
# horizon.
ELEVATION = 34.0

In [13]:
class SunObserver(ephem.Observer):
    """Extends an ephem.Observer object to only observe the sun. Adds
    convenient methods to calculate times and positions of sunsets.
    """
    def __init__(self, lat: str | float, lon: str | float, elevation: float) -> None:
        """Initializes the SunObserver object, setting the sun as a
        private property to be accessed from the added methods.

        Args:
            lat: The observer's latitude.
            lon: The observer's longitude.
            elevation: The observer's elevation. ephem treats this as if the horizon is
                also this elevation, so ensure this is the general elevation of the
                surrounding area, rather than if the observer is standing on a high
                object relative to the horizon.
        """
        super().__init__()
        self.lat = str(lat)
        self.lon = str(lon)
        self.elevation = elevation
        self._sun = ephem.Sun()

    def get_sunset_time_on_date(self, d: datetime) -> datetime:
        self.date = d
        sunset = self.next_setting(self._sun)
        return ephem.localtime(sunset)
    
    def get_sunset_azimuth_on_date(self, d: datetime) -> float:
        # Set observer datetime to sunset time on given date
        self.date = self.get_sunset_time_on_date(d)
        self._sun.compute(self)
        return degrees(self._sun.az)
    
    def get_next_date_with_sunset_azimuth_greater_than(
            self, degrees: float, start_date: datetime = datetime.now()
        ) -> datetime:
        for i in range(365):
            d = start_date + timedelta(days=i)
            az = self.get_sunset_azimuth_on_date(d)
            if az > degrees:
                return self.get_sunset_time_on_date(d)


In [14]:
sun_observer = SunObserver(BALCONY_LAT, BALCONY_LON, ELEVATION)
next_sunset_time = sun_observer.get_sunset_time_on_date(datetime.now())
print(f'The next sunset will be at {next_sunset_time:%Y-%m-%d %H:%M}')

The next sunset will be at 2024-03-24 18:27


In [15]:
next_sunset_degrees = sun_observer.get_sunset_azimuth_on_date(datetime.now())
print(f'The next sunset will take place at {next_sunset_degrees:.0f}° on the horizon.')

The next sunset will take place at 286° on the horizon.


The line of sight from my balcony to the horizon is mostly blocked by a very high wall.
The minimum line of sight is around 310°. I would like to know on which day I can expect
to see the sunset from my balcony.

In [16]:
BALCONY_MIN_AZIMUTH = 310
next_visible_sunset_time = sun_observer.get_next_date_with_sunset_azimuth_greater_than(
    BALCONY_MIN_AZIMUTH
)
print(
    f'The next sunset visible from my balcony will be at '
    f'{next_visible_sunset_time:%Y-%m-%d %H:%M}'
)

The next sunset visible from my balcony will be at 2024-04-10 19:57
